In [4]:
import pandas as pd
import re

# Load the generated CSV

In [38]:
stock_prices = pd.read_csv('/Users/toddtan/Desktop/Records/personal_projects/leetcode/pandas/validate_stock_data/stock_prices_mock.csv')
stock_prices.head(5)

,date,symbol,close
0,2022-01-03,AAPL,99.28
1,2022-01-03,GOOG,149.13
2,2022-01-03,TSLA,201.87
3,2022-01-03,AMZN,250.18
4,2022-01-03,MSFT,293.11


# 1. Check Missing values Summary

In [39]:
stock_prices.isnull().sum()

date       0
symbol     0
close     58
dtype: int64

# 2. Data types

In [40]:
stock_prices.dtypes

date       object
symbol     object
close     float64
dtype: object

# 3. Convert 'date' to datetime and validate

In [41]:
stock_prices['date'] = pd.to_datetime(stock_prices['date'], errors='coerce')
stock_prices.dtypes

date      datetime64[ns]
symbol            object
close            float64
dtype: object

# 4. Validate symbol: uppercase letters only, no special chars

In [45]:
stock_prices[~stock_prices['symbol'].astype(str).str.fullmatch('[A-Z]+')]

,date,symbol,close
388,2022-02-24,bad$symbol,499.71
838,2022-04-28,bad$symbol,496.14
1028,2022-05-25,bad$symbol,499.07
1058,2022-05-30,bad$symbol,498.30
1183,2022-06-16,bad$symbol,257.59
1467,2022-07-26,bad$symbol,448.16
1672,2022-08-24,bad$symbol,205.66
1710,2022-08-30,bad$symbol,92.54
1712,2022-08-30,bad$symbol,203.30
2081,2022-10-20,bad$symbol,151.69


# 5. Validate close price: must be a positive float

In [51]:
stock_prices[stock_prices['close'] <= 0]

,date,symbol,close
1259,2022-06-27,INTC,-548.06
1446,2022-07-22,META,-404.09
1520,2022-08-03,AAPL,-101.15
1608,2022-08-15,BABA,-496.46
1884,2022-09-22,MSFT,-298.92
1910,2022-09-27,AAPL,-102.62
1936,2022-09-29,META,-399.41
2161,2022-11-01,GOOG,-151.75
2617,2023-01-03,NFLX,-453.18
2761,2023-01-24,GOOG,-149.68


# 6. Check for duplicate (date, symbol) combinations

In [59]:
stock_prices[stock_prices.duplicated(subset=['date', 'symbol'], keep=False)]

,date,symbol,close
1710,2022-08-30,bad$symbol,92.54
1712,2022-08-30,bad$symbol,203.30


# 7. Check for outliers in close prices

In [64]:
price_mean = stock_prices['close'].mean(skipna=True)
price_std = stock_prices['close'].std(skipna=True)
outliers = stock_prices[(stock_prices['close'] - price_mean).abs() > 4 * price_std]
outliers

,date,symbol,close
1259,2022-06-27,INTC,-548.06
1446,2022-07-22,META,-404.09
1608,2022-08-15,BABA,-496.46
1884,2022-09-22,MSFT,-298.92
1936,2022-09-29,META,-399.41
2617,2023-01-03,NFLX,-453.18
3264,2023-04-04,MSFT,-303.99
3426,2023-04-26,META,-398.06
4334,2023-08-31,MSFT,-300.25
4385,2023-09-07,NVDA,-352.14


# 8. Validate time series continuity: each symbol should have consistent trading dates

In [73]:
prev = stock_prices['close'].ffill
next_ = stock_prices['close'].bfill

stock_prices['close'] = stock_prices.groupby('symbol')['close'].transform(lambda x: x.interpolate(method='linear'))

date      0
symbol    0
close     0
dtype: int64